In [1]:
import pandas as pd
import numpy as np

In [2]:
combined_df=pd.read_csv("Non vpn dataset update.csv")

In [3]:
row_count, column_count = combined_df.shape
print("Row count:", row_count)
print("Column count:", column_count)


Row count: 199905
Column count: 73


In [4]:
label_distribution = combined_df['Label'].value_counts()
print("\nLabel Distribution:")
print(label_distribution)


Label Distribution:
Label
VOIP             122579
EMAIL             56159
CHAT              17755
FILE TRANSFER      2738
STREAMING           674
Name: count, dtype: int64


In [5]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding, SpatialDropout1D,Dropout
from keras.preprocessing.text import Tokenizer
from keras.utils.np_utils import to_categorical

In [6]:
# Assuming combined_df is your DataFrame containing the data
voip_packets = combined_df[combined_df['Label'] == 'VOIP'].head(5000)
email_packets = combined_df[combined_df['Label'] == 'EMAIL'].head(5000)
chat_packets = combined_df[combined_df['Label'] == 'CHAT'].head(5000)
remaining_packets = combined_df[(combined_df['Label'] != 'VOIP') & (combined_df['Label'] != 'EMAIL') & (combined_df['Label'] != 'CHAT') ]

# Concatenate the first 2000 VOIP packets, 2000 P2P packets, and the remaining packets
final_dataset = pd.concat([voip_packets, email_packets,chat_packets, remaining_packets], ignore_index=True)
print(final_dataset.shape)


(18412, 73)


In [7]:
unwanted_columns = ['frame.len','tcp.flags.syn','tcp.flags.ack','tcp.flags.push', 'tcp.flags.reset',\
                   'data.len']  # Add all unwanted column names here
# Remove unwanted columns from the DataFrame
filtered_df = final_dataset.drop(columns=unwanted_columns)

# Display the filtered DataFrame
print(filtered_df)


       ip.proto  tcp.srcport  tcp.dstport  udp.srcport  udp.dstport  tcp.seq   
0          17.0          NaN          NaN      19305.0      33306.0      NaN  \
1          17.0          NaN          NaN      19305.0      33306.0      NaN   
2          17.0          NaN          NaN      19305.0      33306.0      NaN   
3          17.0          NaN          NaN      19305.0      33306.0      NaN   
4          17.0          NaN          NaN      19305.0      33306.0      NaN   
...         ...          ...          ...          ...          ...      ...   
18407      17.0          NaN          NaN        137.0        137.0      NaN   
18408      17.0          NaN          NaN      61009.0      45087.0      NaN   
18409      17.0          NaN          NaN      45087.0      61009.0      NaN   
18410      17.0          NaN          NaN      35468.0      61009.0      NaN   
18411      17.0          NaN          NaN      61009.0      35468.0      NaN   

       frame.time_epoch  tcp.stream  ip

In [8]:
# Create a LabelEncoder object
label_encoder = LabelEncoder()

# Fit and transform the 'Label' column
filtered_df['Label'] = label_encoder.fit_transform(filtered_df['Label'])

 # Display the encoded labels
print("Encoded Labels:")
print(filtered_df['Label'].value_counts())


Encoded Labels:
Label
4    5000
1    5000
0    5000
2    2738
3     674
Name: count, dtype: int64


In [9]:
# Mapping original labels to encoded values
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print(label_mapping)

{'CHAT': 0, 'EMAIL': 1, 'FILE TRANSFER': 2, 'STREAMING': 3, 'VOIP': 4}


In [10]:
X = filtered_df.drop(columns=['Label'])
y = filtered_df['Label']


In [11]:
encoder = LabelEncoder()
y = encoder.fit_transform(y)
num_classes = len(encoder.classes_)


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape)
print(y_train.shape)

(14729, 66)
(14729,)


In [13]:
from sklearn.impute import SimpleImputer

# Impute missing values in the features
imputer = SimpleImputer(strategy='mean')
X_train = imputer.fit_transform(X_train)
X_test= imputer.transform(X_test)

In [14]:
x_training_data = np.reshape(X_train, (X_train.shape[0], X_train.shape[1],1))
print(x_training_data.shape)


(14729, 50, 1)


In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, SimpleRNN, Dropout

# Define input shape based on your data
input_shape = (x_training_data.shape[1], 1)  # Shape of your input data assuming it's sequential

# Define the number of classes for the output
num_classes = 5  # Assuming you have 6 classes for the output

# Create the combined RNN-CNN model
model = Sequential([
    # CNN layers
    Conv1D(32, 3, activation='relu', input_shape=input_shape),
    MaxPooling1D(2),
    Conv1D(64, 3, activation='relu'),
    MaxPooling1D(2),
    #Flatten(),

    # RNN layers
    SimpleRNN(units=64, return_sequences=True),
    Dropout(0.2),
    SimpleRNN(units=64),
    Dropout(0.2),

    # Output layer
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Print the model summary
model.summary()

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=24, validation_data=(X_test, y_test))


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 48, 32)            128       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 24, 32)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 22, 64)            6208      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 11, 64)           0         
 1D)                                                             
                                                                 
 simple_rnn (SimpleRNN)      (None, 11, 64)            8256      
                                                                 
 dropout (Dropout)           (None, 11, 64)            0

Epoch 46/100
614/614 [==============================] - 5s 8ms/step - loss: 0.3123 - accuracy: 0.8882 - val_loss: 0.2868 - val_accuracy: 0.8938
Epoch 47/100
614/614 [==============================] - 5s 8ms/step - loss: 0.3099 - accuracy: 0.8872 - val_loss: 0.2994 - val_accuracy: 0.8917
Epoch 48/100
614/614 [==============================] - 5s 8ms/step - loss: 0.3153 - accuracy: 0.8849 - val_loss: 0.2875 - val_accuracy: 0.8944
Epoch 49/100
614/614 [==============================] - 5s 8ms/step - loss: 0.3269 - accuracy: 0.8831 - val_loss: 0.2965 - val_accuracy: 0.8985
Epoch 50/100
614/614 [==============================] - 5s 8ms/step - loss: 0.3086 - accuracy: 0.8929 - val_loss: 0.2886 - val_accuracy: 0.9052
Epoch 51/100
614/614 [==============================] - 5s 8ms/step - loss: 0.3058 - accuracy: 0.8893 - val_loss: 0.2772 - val_accuracy: 0.8982
Epoch 52/100
614/614 [==============================] - 5s 8ms/step - loss: 0.3298 - accuracy: 0.8791 - val_loss: 0.3175 - val_accuracy:

In [20]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Define input shape based on your data
input_shape = (x_training_data.shape[1], 1)  # Shape of your input data

# Define the number of classes for the output
num_classes = 5  # Assuming you have 6 classes for the output

# Create the RNN model
rnn_model = Sequential([
    SimpleRNN(units=64, return_sequences=True, input_shape=input_shape),
    Dropout(0.2),
    SimpleRNN(units=64, return_sequences=True),
    Dropout(0.2),
    SimpleRNN(units=64),
    Dropout(0.2),
    Dense(units=num_classes, activation='softmax')
])

# Compile the model
rnn_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Print the model summary
rnn_model.summary()

# Train the model with early stopping
history = rnn_model.fit(X_train, y_train, epochs=75, batch_size=32,
                        validation_data=(X_test, y_test), callbacks=[early_stopping])


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_5 (SimpleRNN)    (None, 50, 64)            4224      
                                                                 
 dropout_5 (Dropout)         (None, 50, 64)            0         
                                                                 
 simple_rnn_6 (SimpleRNN)    (None, 50, 64)            8256      
                                                                 
 dropout_6 (Dropout)         (None, 50, 64)            0         
                                                                 
 simple_rnn_7 (SimpleRNN)    (None, 64)                8256      
                                                                 
 dropout_7 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 5)                

In [17]:
# Evaluate the model
score = model.evaluate(X_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.2919861972332001
Test accuracy: 0.8919359445571899


In [21]:
# Evaluate the model
score = rnn_model.evaluate(X_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.24927349388599396
Test accuracy: 0.9266902208328247


In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Create a Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier
rf_classifier.fit(X_train, y_train)

# Predict on the test set
y_pred = rf_classifier.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.9994569644311703
